# Анализ эксперимента run-pid-v2

Анализ работы инкрементального ПИД-регулятора на реальной установке. 

Запуски:
* 2026-02-09_17-09-15
* 2026-02-09_17-49-22

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from nn_laser_stabilizer.config.config import load_config
from nn_laser_stabilizer.utils.paths import get_experiment_dir

## Загрузка эксперимента

In [ ]:
EXPERIMENT_NAME = "run-pid-v2"
EXPERIMENT_DATE = "2026-02-09"
EXPERIMENT_TIME = "17-09-15"

EXPERIMENT_DIR = get_experiment_dir(
    experiment_name=EXPERIMENT_NAME, 
    experiment_date=EXPERIMENT_DATE, 
    experiment_time=EXPERIMENT_TIME)

print(f"Директория эксперимента: {EXPERIMENT_DIR}")
print(f"Существует: {EXPERIMENT_DIR.exists()}")


In [ ]:
config = load_config(EXPERIMENT_DIR / "config.yaml")
pid_config = config.pid

print(f"Эксперимент: {config.experiment_name}")
print(f"\nПараметры PID:")
print(f"  kp = {pid_config.kp}")
print(f"  ki = {pid_config.ki}")
print(f"  kd = {pid_config.kd}")
print(f"  dt = {pid_config.dt}")
print(f"\nУставка: {config.setpoint}")
print(f"Warmup: {config.warmup_steps} шагов, output={config.warmup_output}")

## Парсинг логов

In [ ]:
import json

log_path = EXPERIMENT_DIR / "pid_data.jsonl"

records = []
with open(log_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:
            records.append(json.loads(line))

df = pd.DataFrame(records)

df["setpoint"] = config.setpoint
df["error"] = df["process_variable"] - df["setpoint"]

print(f"Загружено {len(df)} шагов")
df.head(10)

## Обзор данных

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df["step"], df["process_variable"], label="Process Variable", alpha=0.8, linewidth=0.5)
ax.axhline(config.setpoint, color="red", linestyle="--", label=f"Setpoint = {config.setpoint}")
ax.set_ylabel("Process Variable")
ax.set_xlabel("Step")
ax.legend(loc="upper right")
ax.set_title("Process Variable vs Setpoint")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df["step"], df["control_output"], label="Control Output", alpha=0.8, linewidth=0.5, color="green")
ax.set_ylabel("Control Output")
ax.set_xlabel("Step")
ax.legend(loc="upper right")
ax.set_title("Control Output")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df["step"], df["error"], label="Error", alpha=0.8, linewidth=0.5, color="purple")
ax.axhline(0, color="red", linestyle="--", alpha=0.5)
ax.set_ylabel("Error (PV - SP)")
ax.set_xlabel("Step")
ax.legend(loc="upper right")
ax.set_title("Error")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(df["step"], df["delta"], label="Delta", alpha=0.8, linewidth=0.5)
ax.set_ylabel("Delta")
ax.set_xlabel("Step")
ax.legend(loc="upper right")
ax.set_title("Delta")
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Симуляция системы

In [ ]:
import numpy as np

mask = df["is_warming_up"] | (~df["is_warming_up"])
pv = df.loc[mask, "process_variable"].values.astype(np.float64) / 10
co = df.loc[mask, "control_output"].values.astype(np.float64)

na = 1  # лаги pv
nb = 2  # лаги co
nk = 0  # задержка co

offset = max(na, nb + nk - 1)
N = len(pv)

cols = []
for i in range(1, na + 1):
    cols.append(pv[offset - i : N - i])
for i in range(nb):
    cols.append(co[offset - nk - i : N - nk - i])

X = np.column_stack(cols)
y = pv[offset:N]

theta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
y_pred = X @ theta

a_coeffs = theta[:na]
b_coeffs = theta[na:]

rmse = np.sqrt(np.mean((y - y_pred) ** 2))
print(f"ARX(na={na}, nb={nb}, nk={nk})")
for i, a in enumerate(a_coeffs):
    print(f"  a{i+1} = {a:.4f}")
for i, b in enumerate(b_coeffs):
    print(f"  b{i+1} = {b:.4f}")
print(f"RMSE: {rmse:.2f}")

den = [1.0] + [-a for a in a_coeffs]
num = [0.0] * nk + list(b_coeffs)
poles = np.roots(den)
print(f"Полюса: {poles}")
print(f"Устойчива: {all(np.abs(poles) < 1)}")

plt.figure(figsize=(12, 4))
plt.plot(y, label="Реальное PV")
plt.plot(y_pred, label="ARX модель", linestyle="--")
plt.legend()
plt.title(f"ARX({na},{nb},{nk}) | RMSE={rmse:.2f}")
plt.xlim(0, 1000)
plt.ylim(100, 150)
plt.xlabel("Шаг")
plt.ylabel("process_variable")
plt.show()

In [ ]:
plt.figure(figsize=(12, 3))
plt.plot(y - y_pred, color="red", alpha=0.7)
plt.axhline(0, color="black", linewidth=0.5)
plt.ylabel("Ошибка")
plt.xlabel("Шаг")
plt.title(f"Ошибка модели ARX({na},{nb},{nk})")
plt.xlim(2500, 2700)
plt.ylim(-5, 5)
plt.tight_layout()
plt.show()

In [ ]:
na = 1
nb = 2
nk = 0

offset = max(na, nb + nk - 1)
N = len(pv)

cols = []
for i in range(1, na + 1):
    cols.append(pv[offset - i : N - i])
for i in range(nb):
    cols.append(co[offset - nk - i : N - nk - i])
cols.append(np.ones(N - offset))  # свободный член

X = np.column_stack(cols)
y = pv[offset:N]

theta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
a1 = theta[0]
b1, b2 = theta[1], theta[2]
c0 = theta[3]
print(f"a1={a1:.4f}, b1={b1:.4f}, b2={b2:.4f}, c0={c0:.4f}")

In [ ]:
pv_sim = np.zeros(N)
pv_sim[0] = pv[0]

for t in range(1, N):
    pv_sim[t] = a1 * pv_sim[t - 1] + b1 * co[t] + b2 * co[t - 1] + c0

plt.figure(figsize=(12, 4))
plt.plot(pv, label="Реальное PV")
plt.plot(pv_sim, label="Симуляция", linestyle="--")
plt.legend()
plt.xlabel("Шаг")
plt.ylabel("process_variable")
plt.title("Свободная симуляция ARX с константой")
plt.show()

rmse_sim = np.sqrt(np.mean((pv - pv_sim) ** 2))
print(f"RMSE симуляции: {rmse_sim:.4f}")

In [ ]:
residuals = y - y_pred

print(f"Mean: {residuals.mean():.2f}")
print(f"STD:     {residuals.std():.2f}")

noise_std = residuals.std()

freqs = np.fft.rfftfreq(len(residuals), d=0.005)
spectrum = np.abs(np.fft.rfft(residuals))

plt.figure(figsize=(12, 3))
plt.plot(freqs[1:], spectrum[1:])
plt.xlabel("Частота, Гц")
plt.ylabel("Амплитуда")
plt.title("Спектр возмущений")
plt.show()

In [ ]:
from scipy.signal import find_peaks

dt = 0.005
freqs = np.fft.rfftfreq(len(residuals), d=dt)
spectrum = np.abs(np.fft.rfft(residuals))

peaks, props = find_peaks(spectrum[1:], prominence=spectrum[1:].max() * 0.1)
peaks += 1

top_n = 3
top_idx = np.argsort(spectrum[peaks])[-top_n:][::-1]
peaks = peaks[top_idx]

peak_freqs = freqs[peaks]
peak_amps = spectrum[peaks] * 2 / len(residuals)

for i, (f, a) in enumerate(zip(peak_freqs, peak_amps)):
    print(f"Пик {i+1}: частота = {f:.2f} Гц, амплитуда = {a:.2f}")

plt.figure(figsize=(12, 3))
plt.plot(freqs[1:], spectrum[1:])
plt.plot(peak_freqs, spectrum[peaks], "rx", markersize=10)
for f, a in zip(peak_freqs, spectrum[peaks]):
    plt.annotate(f"{f:.1f} Гц", (f, a), textcoords="offset points", xytext=(5, 10))
plt.xlabel("Частота, Гц")
plt.ylabel("Амплитуда")
plt.title("Спектр возмущений")
plt.show()

In [ ]:
f1, amp1 = peak_freqs[0], peak_amps[0]
f2, amp2 = peak_freqs[1], peak_amps[1]

print(f"f1 = {f1:.2f} Гц, amp1 = {amp1:.4f}")
print(f"f2 = {f2:.2f} Гц, amp2 = {amp2:.4f}")

In [ ]:
na = 1
nb = 2
nk = 0

offset = max(na, nb + nk - 1)
N = len(pv)

cols = []
for i in range(1, na + 1):
    cols.append(pv[offset - i : N - i])
for i in range(nb):
    cols.append(co[offset - nk - i : N - nk - i])
cols.append(np.ones(N - offset))

t_arr = np.arange(offset, N) * dt
for f in [f1, f2]:
    cols.append(np.sin(2 * np.pi * f * t_arr))
    cols.append(np.cos(2 * np.pi * f * t_arr))

X = np.column_stack(cols)
y = pv[offset:N]

theta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
y_pred = X @ theta

a1 = theta[0]
b1, b2 = theta[1], theta[2]
c0 = theta[3]
s1, c1_coef = theta[4], theta[5]  
s2, c2_coef = theta[6], theta[7]  

# Амплитуда и фаза из sin+cos
amp1_fit = np.sqrt(s1**2 + c1_coef**2)
amp2_fit = np.sqrt(s2**2 + c2_coef**2)
phi1_fit = np.arctan2(c1_coef, s1)
phi2_fit = np.arctan2(c2_coef, s2)

print(f"a1={a1:.4f}, b1={b1:.4f}, b2={b2:.4f}, c0={c0:.4f}")
print(f"f1={f1:.2f} Гц, amp={amp1_fit:.4f}, phi={np.degrees(phi1_fit):.1f}°")
print(f"f2={f2:.2f} Гц, amp={amp2_fit:.4f}, phi={np.degrees(phi2_fit):.1f}°")

rmse = np.sqrt(np.mean((y - y_pred) ** 2))
print(f"RMSE: {rmse:.4f}")

In [ ]:
pv_sim = np.zeros(N)
pv_sim[0] = pv[0]

for t in range(1, N):
    time = t * dt
    disturbance = (amp1_fit * np.sin(2 * np.pi * f1 * time + phi1_fit)
                 + amp2_fit * np.sin(2 * np.pi * f2 * time + phi2_fit))
    pv_sim[t] = a1 * pv_sim[t - 1] + b1 * co[t] + b2 * co[t - 1] + c0 + disturbance

plt.figure(figsize=(12, 4))
plt.plot(pv, label="Реальное PV")
plt.plot(pv_sim, label="Симуляция с возмущениями", linestyle="--")
plt.legend()
plt.xlabel("Шаг")
plt.ylabel("process_variable")
plt.title("Симуляция ARX + периодические возмущения")
plt.show()

rmse_sim = np.sqrt(np.mean((pv - pv_sim) ** 2))
print(f"RMSE симуляции: {rmse_sim:.4f}")

In [ ]:
a1 = 0.9360
b1 = 0.2205
b2 = -0.2205
c0 = 8.2604
f1_freq, amp1_fit, phi1_fit = 53.49, 0.0603, np.radians(70.4)
f2_freq, amp2_fit, phi2_fit = 29.78, 0.0530, np.radians(29.6)

kp, ki, kd = 3.5, 11.0, 0.002
pid_dt = 0.005
setpoint = 70

N_sim = 10_000
pv_sim = np.zeros(N_sim)
co_sim = np.zeros(N_sim)
pv_sim[0] = 100.0
co_sim[0] = 2000

integral = 0.0
prev_error = 0.0
reset_counter = 0

for t in range(1, N_sim):
    time = t * pid_dt
    disturbance = (amp1_fit * np.sin(2 * np.pi * f1_freq * time + phi1_fit)
                 + amp2_fit * np.sin(2 * np.pi * f2_freq * time + phi2_fit))

    if reset_counter > 0:
        co_sim[t] = 2000
        reset_counter -= 1
        if reset_counter == 0:
            integral = 0.0
            prev_error = 0.0
    else:
        error = setpoint - pv_sim[t - 1]
        integral += error * pid_dt
        derivative = (error - prev_error) / pid_dt
        delta_co = kp * error + ki * integral + kd * derivative
        prev_error = error

        co_sim[t] = np.clip(co_sim[t - 1] + delta_co, -np.inf, np.inf)

        # if co_sim[t] <= 0 or co_sim[t] >= 4095:
        #     reset_counter = 1000
        #     co_sim[t] = 2000
        #     integral = 0.0
        #     prev_error = 0.0

    pv_sim[t] = (a1 * pv_sim[t - 1]
               + b1 * co_sim[t] + b2 * co_sim[t - 1]
               + c0 + disturbance
               + np.random.normal(0, noise_std))

# Графики
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

axes[0].plot(pv_sim)
axes[0].axhline(setpoint, color="red", linestyle="--", label=f"setpoint={setpoint}")
axes[0].set_ylabel("process_variable")
axes[0].legend()
axes[0].set_title("PID на модели ARX")

axes[1].plot(co_sim)
axes[1].set_ylabel("control_output")

axes[2].plot(setpoint - pv_sim, color="red", alpha=0.7)
axes[2].axhline(0, color="black", linewidth=0.5)
axes[2].set_ylabel("Ошибка")
axes[2].set_xlabel("Шаг")

plt.tight_layout()
plt.show()

print(f"Средняя ошибка: {np.mean(np.abs(setpoint - pv_sim[100:])):.4f}")
print(f"Макс ошибка:    {np.max(np.abs(setpoint - pv_sim[100:])):.4f}")